# 1 Priemra pagina IEEE

In [1]:
#imports requeridos para que el proyecto funcione
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import time
import os

import os
from dotenv import load_dotenv

# Cargar variables del archivo .env
load_dotenv()


# Obtener el correo electrónico
email = os.getenv("EMAIL")
# Obtener la contraseña
password = os.getenv("PASSWORD")



In [2]:
# Configurar la ruta de descarga (usando una ruta absoluta)
download_path = os.getenv("DOWNLOAD_PATH")  # Cambia esto a tu ruta real
os.makedirs(download_path, exist_ok=True)  # Crear la carpeta si no existe

chrome_options = webdriver.ChromeOptions()


# Configuración de preferencias de descargas
prefs = {
    "download.default_directory": download_path,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": False,  # Desactivar SafeBrowsing puede ayudar
    "profile.default_content_settings.popups": 0,
    "profile.content_settings.exceptions.automatic_downloads.*.setting": 1
}
chrome_options.add_experimental_option("prefs", prefs)

# Argumentos adicionales que pueden ayudar
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-download-notification")
chrome_options.add_argument("--disable-popup-blocking")



In [3]:
# Iniciar el navegador
driver = webdriver.Chrome(options=chrome_options)

# Abrir la URL
url1 = 'https://login.intelproxy.com/v2/inicio?cuenta=7Ah6RNpGWF22jjyq'
driver.get(url1)

In [4]:
login_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/div/div/div/div[1]/div[2]/a"))
)
login_button.click()

In [5]:
email_input = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "identifierId"))
)

email_input.send_keys(email)
email_input.send_keys(Keys.RETURN)

In [6]:
password_input = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.NAME, "Passwd"))
)# Nombre del campo de contraseña
password_input.send_keys(password)
password_input.send_keys(Keys.RETURN)

In [7]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Buscar el botón <summary> que contiene "Fac. Ingeniería"
boton = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.XPATH, "//summary[contains(., 'Fac. Ingeniería')]"))
)

time.sleep(2)
boton.click()

AttributeError: 'NoneType' object has no attribute 'is_displayed'

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Buscar el enlace a IEEE por su XPath correcto
enlaceIEEE = WebDriverWait(driver, 15).until(
    EC.element_to_be_clickable((By.XPATH, "//*[@id='facingenieraieeeinstituteofelectricalandelectronicsengineersdescubridor']//h2[@class='result-title']/a"))
)


In [ ]:
# Hacer clic en el enlace
enlaceIEEE.click()

In [ ]:
try:
    # Esperar a que aparezca cualquiera de los botones
    boton_cookies = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "button.osano-cm-dialog__close, button.osano-cm-denyAll, button.osano-cm-accept-all"))
    )
    driver.execute_script("arguments[0].click();", boton_cookies)
    print("🍪 Banner de cookies cerrado.")
except Exception as e:
    print(f"⚠ No apareció el banner de cookies o ya estaba cerrado: {e}")


🍪 Banner de cookies cerrado.


In [ ]:
# Esperar a que la barra de búsqueda esté clickeable y visible
barra_busqueda = WebDriverWait(driver, 15).until(
    EC.element_to_be_clickable((By.XPATH, "//*[@id='LayoutWrapper']/div/div/div[3]/div/xpl-root/header/xpl-header/div/div[2]/div[2]/xpl-search-bar-migr/div/form/div[2]/div/div[1]/xpl-typeahead-migr/div/input"))
)

# Hacer scroll al elemento para asegurarse de que esté visible
driver.execute_script("arguments[0].scrollIntoView(true);", barra_busqueda)
time.sleep(1)

# Hacer clic primero para asegurarse de que está activo
barra_busqueda.click()
time.sleep(0.5)

In [ ]:
# Escribir "generative artificial intelligence" entre comillas para reducir el rango de busqueda
barra_busqueda.clear()  # Limpiar cualquier texto previo
barra_busqueda.send_keys('generative artificial intelligence')
time.sleep(1)  # Pequeña pausa para asegurar que el texto se escribió

In [ ]:
# Esperar a que el botón de búsqueda esté clickeable
boton_buscar = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//*[@id='LayoutWrapper']/div/div/div[3]/div/xpl-root/header/xpl-header/div/div[2]/div[2]/xpl-search-bar-migr/div/form/div[2]/div/div[2]/button"))
)

In [ ]:
boton_buscar.click()

In [ ]:
# Función para verificar que la descarga se completó
def esperar_descarga(carpeta, tiempo_max=30):
    """
    Espera hasta que aparezca un nuevo archivo .bib en la carpeta de descargas
    o hasta que se agote el tiempo máximo.
    """
    inicio = time.time()
    archivos_iniciales = set([f for f in os.listdir(carpeta) if f.endswith('.bib')])
    
    while time.time() - inicio < tiempo_max:
        time.sleep(2)  # Comprobar cada 2 segundos
        archivos_actuales = set([f for f in os.listdir(carpeta) if f.endswith('.bib')])
        nuevos_archivos = archivos_actuales - archivos_iniciales
        
        if nuevos_archivos:
            # Encontró nuevo archivo
            nuevo_archivo = list(nuevos_archivos)[0]
            return os.path.join(carpeta, nuevo_archivo)
        
    print("⚠️ Tiempo de espera de descarga agotado.")
    return None


In [ ]:
try:
    # Esperar a que aparezca el botón de aceptar/cerrar cookies
    boton_cookies = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div[2]/div[2]/button[2]"))  # Usa el XPath correcto
    )
    boton_cookies.click()
    print("Se cerró la ventana de cookies.")
except:
    print("⚠ No apareció la ventana de cookies o ya estaba cerrada.")

⚠ No apareció la ventana de cookies o ya estaba cerrada.


In [ ]:
# 1. Hacer clic en el menú "Items Per Page"
itemsPerPage_link = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, 'dropdownPerPageLabel'))
)
itemsPerPage_link.click()

# 2. Seleccionar la opción deseada (ejemplo: 100)
option_100 = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), '100')]"))
)
option_100.click()


In [ ]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

# 📌 Función auxiliar para esperar descargas nuevas
def esperar_descarga(download_path, archivos_antes, timeout=30):
    """
    Espera a que aparezca un nuevo archivo .bib en la carpeta de descargas
    que no esté en la lista anterior.
    """
    end_time = time.time() + timeout
    while time.time() < end_time:
        archivos_actuales = set(f for f in os.listdir(download_path) if f.endswith(".bib"))
        nuevos = archivos_actuales - archivos_antes
        if nuevos:
            nuevo_archivo = max([os.path.join(download_path, f) for f in nuevos],
                              key=os.path.getctime)
            return nuevo_archivo
        time.sleep(1)
    return None

# 📌 Configura aquí tu carpeta de descargas base
base_download_path = os.getenv("DOWNLOAD_PATH")

# 📌 Crear carpeta ieee dentro de descargas
ieee_folder = os.path.join(base_download_path, "ieee")
if not os.path.exists(ieee_folder):
    os.makedirs(ieee_folder)
    print(f"✅ Carpeta creada: {ieee_folder}")
else:
    print(f"📁 Usando carpeta existente: {ieee_folder}")

# Usar la carpeta ieee como directorio de trabajo
download_path = ieee_folder

# Inicializar el contador de páginas
page_num = 1
hay_mas_paginas = True

# Bucle para navegar a través de todas las páginas
while hay_mas_paginas:
    print(f"\nProcesando página {page_num}...")

    time.sleep(2)  # pequeña espera entre páginas

    # Seleccionar checkbox
    try:
        checkbox = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH,
                '//*[@id="xplMainContent"]/div[2]/div[2]/xpl-results-list/div[2]/label/input'))
        )
        if not checkbox.is_selected():
            driver.execute_script("arguments[0].click();", checkbox)
        time.sleep(1)
    except Exception as e:
        print(f"⚠️ Error al seleccionar checkbox en página {page_num}: {e}")
        continue

    # Botón Export
    try:
        enlace = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH,
                "//*[@id='xplMainContent']/div[1]/div[1]/ul/li[3]/xpl-export-search-results/button"))
        )
        driver.execute_script("arguments[0].click();", enlace)
        time.sleep(2)
    except Exception as e:
        print(f"⚠️ Error al hacer clic en export en página {page_num}: {e}")
        continue

    # Opción Citations
    try:
        all_results = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH,
                "/html/body/ngb-modal-window/div/div/div[1]/ul/li[2]/a"))
        )
        all_results.click()
        time.sleep(2)
    except Exception as e:
        print(f"⚠️ Error al seleccionar Citations en página {page_num}: {e}")
        continue

    # Seleccionar BibTeX
    try:
        bibtex_link = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH,
                "/html/body/ngb-modal-window/div/div/div[2]/div/xpl-citation-download/form/div[1]/section[1]/div/label[2]/input"))
        )
        bibtex_link.click()
        time.sleep(1)
    except Exception as e:
        print(f"⚠️ Error al seleccionar BibTeX en página {page_num}: {e}")
        continue

    # Citation + Abstract
    try:
        citAndAbstract_link = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH,
                "/html/body/ngb-modal-window/div/div/div[2]/div/xpl-citation-download/form/div[1]/section[2]/div/label[2]/input"))
        )
        citAndAbstract_link.click()
        time.sleep(1)
    except Exception as e:
        print(f"⚠️ Error al seleccionar Citation and Abstract en página {page_num}: {e}")
        continue

    # Guardar estado de descargas antes de bajar
    archivos_antes = set([f for f in os.listdir(download_path) if f.endswith(".bib")])

    # Botón descargar
    try:
        download_button = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH,
                "/html/body/ngb-modal-window/div/div/div[2]/div/xpl-citation-download/form/div[2]/button[2]"))
        )
        download_button.click()
        print(f"⬇️ Intentando descarga para página {page_num}...")

        # Nota: El archivo se descargará primero en base_download_path
        # y luego lo moveremos a ieee_folder
        archivos_antes_base = set([f for f in os.listdir(base_download_path) if f.endswith(".bib")])
        
        # Esperar el archivo en la carpeta base de descargas
        end_time = time.time() + 50
        nuevo_archivo = None
        while time.time() < end_time:
            archivos_actuales = set(f for f in os.listdir(base_download_path) if f.endswith(".bib"))
            nuevos = archivos_actuales - archivos_antes_base
            if nuevos:
                archivo_descargado = max([os.path.join(base_download_path, f) for f in nuevos],
                                  key=os.path.getctime)
                # Mover a la carpeta ieee
                nuevo_nombre = f"ieee_generative_ai_page_{page_num}.bib"
                ruta_nueva = os.path.join(download_path, nuevo_nombre)
                os.rename(archivo_descargado, ruta_nueva)
                print(f"✅ Archivo guardado como: {nuevo_nombre}")
                nuevo_archivo = ruta_nueva
                break
            time.sleep(1)
        
        if not nuevo_archivo:
            print(f"❌ No se detectó descarga en página {page_num}")

    except Exception as e:
        print(f"⚠️ Error en el proceso de descarga en página {page_num}: {e}")

    # Cerrar modal
    try:
        cerrarVentana_link = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH,
                "/html/body/ngb-modal-window/div/div/div[1]/div/i"))
        )
        cerrarVentana_link.click()
    except:
        ActionChains(driver).send_keys(Keys.ESCAPE).perform()

    # Verificar si hay más páginas
    try:
        # Intentar encontrar el botón de siguiente página
        siguiente_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 
                'button[class^="stats-Pagination_arrow_next_"], '  # Botón de siguiente
                'button[aria-label="Next page of search results"], '  # Alternativa 1
                'button.next, '  # Alternativa 2
                'a.next, '  # Alternativa 3
                'li.next-btn button'  # Alternativa 4
            ))
        )
        
        # Hacer clic en el botón de siguiente página
        driver.execute_script("arguments[0].click();", siguiente_btn)
        page_num += 1
        print(f"➡️ Avanzando a la página {page_num}...")
        time.sleep(4)  # Esperar a que cargue la nueva página
        
    except Exception as e:
        print("No se pudo encontrar el botón de siguiente página.")
        print(f"Error: {str(e)}")
        hay_mas_paginas = False

print(f"\n🎉 ¡Proceso completado! Se han procesado {page_num} páginas en total.")
print(f"📁 Archivos guardados en: {download_path}")

📁 Usando carpeta existente: /home/yep/Documentos/proyectoAnalisisAlgoritmos/descargas/ieee

Procesando página 1...
⬇️ Intentando descarga para página 1...
✅ Archivo guardado como: ieee_generative_ai_page_1.bib
➡️ Avanzando a la página 2...

Procesando página 2...
⬇️ Intentando descarga para página 2...
✅ Archivo guardado como: ieee_generative_ai_page_2.bib
➡️ Avanzando a la página 3...

Procesando página 3...
⬇️ Intentando descarga para página 3...
✅ Archivo guardado como: ieee_generative_ai_page_3.bib
➡️ Avanzando a la página 4...

Procesando página 4...
⬇️ Intentando descarga para página 4...
✅ Archivo guardado como: ieee_generative_ai_page_4.bib
➡️ Avanzando a la página 5...

Procesando página 5...
⬇️ Intentando descarga para página 5...
✅ Archivo guardado como: ieee_generative_ai_page_5.bib
➡️ Avanzando a la página 6...

Procesando página 6...
⬇️ Intentando descarga para página 6...
✅ Archivo guardado como: ieee_generative_ai_page_6.bib
➡️ Avanzando a la página 7...

Procesando pág

KeyboardInterrupt: 

In [ ]:
driver.quit()